In [1]:
import time
import numpy as np
import pandas as pd
from functions import *

In [2]:
split_data_tp1_app = open_file('../data/data_tp1_app.txt')
split_data_tp1_dec = open_file('../data/data_tp1_dec.txt')

split_data_tp2_app = open_file('../data/data_tp2_app.txt')
split_data_tp2_dec = open_file('../data/data_tp2_dec.txt')

split_data_tp3_app = open_file('../data/data_tp3_app.txt')
split_data_tp3_dec = open_file('../data/data_tp3_dec.txt')

In [42]:
def create_hyperplans_classes(train_data):
    hyper_plans_classes = []
    classes = get_unique_class_num(train_data)
    for i in range(len(classes)):
        for j in range(i+1, len(classes)):
            hyper_plans_classes.append((classes[i], classes[j]))
    return hyper_plans_classes

def predict(weights, inputs):
    weight_sum = np.dot(inputs, weights[1:]) + weights[0]
    if weight_sum > 0:
        prediction = 1
    else:
        prediction = 0            
    return prediction

def linear_perceptron_two_classes(train_data, learning_rate = 0.2, n = 100):
    weights = np.zeros(len(train_data.columns))
    for epoch in range(n):
        sum_error = 0.0
        for index, row in train_data.iterrows():
            temp_line = row.tolist()
            line = list(map(float, temp_line))
            prediction = predict(weights, line[1:])
            error = line[0] - prediction
            sum_error += error**2
            for i in range(len(line[1:])):
                weights[i+1] += learning_rate * error * line[i+1]
            weights[0] += learning_rate * error
        print("Epoch : {}, error = {}".format(epoch+1, sum_error))
    return weights

def linear_train(train_data, learning_rate = 0.2, n = 100):
    hyper_plan_list = create_hyperplans_classes(train_data)
    data_pd = pd.DataFrame(train_data)
    for hyper_plan in hyper_plan_list:

        hyperplan_data = data_pd[(data_pd[0] == hyper_plan[0]) | (data_pd[0] == hyper_plan[1])]
        hyperplan_data.iloc[(hyperplan_data[0] == hyper_plan[0]), 0] = 0
        hyperplan_data.iloc[(hyperplan_data[0] == hyper_plan[1]), 0] = 1
        weights = linear_perceptron_two_classes(hyperplan_data, learning_rate, n)
        print("For the hyperplan : {}, the weights are : {}".format(hyper_plan, weights))


linear_train(split_data_tp1_app, 0.02, 5)

<ipython-input-42-5bcd1520d62f>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hyperplan_data.iloc[(hyperplan_data[0] == hyper_plan[0]), 0] = 0
<ipython-input-42-5bcd1520d62f>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hyperplan_data.iloc[(hyperplan_data[0] == hyper_plan[1]), 0] = 1
Epoch : 1, error = 1.0
Epoch : 2, error = 3.0
Epoch : 3, error = 0.0
Epoch : 4, error = 0.0
Epoch : 5, error = 0.0
For the hyperplan : ('1', '2'), the weights are : [ 0.          0.25055288 -0.02368656]
Epoch : 1, error = 1.0
Epoch : 2, error = 0.0
Epoch : 3, error = 0.0
Epoch : 4, error = 0.0
Epoch : 5, error = 0.0
For the hyperp

In [27]:
# Make a prediction with weights
def predict(row, weights):
    activation = weights[0]
    for i in range(len(row)-1):
        activation += weights[i + 1] * row[i]
    return 1.0 if activation >= 0.0 else 0.0

# Estimate Perceptron weights using stochastic gradient descent
def train_weights(train, l_rate, n_epoch):
    weights = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0.0
        for row in train:
            prediction = predict(row, weights)
            error = row[-1] - prediction
            sum_error += error**2
            weights[0] = weights[0] + l_rate * error
            print(row)
            for i in range(len(row)-1):
                print(row[i])
            weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
            print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
    return weights

# Calculate weights
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
l_rate = 0.1
n_epoch = 5
weights = train_weights(dataset, l_rate, n_epoch)
print(weights)

[2.7810836, 2.550537003, 0]
2.7810836
2.550537003
>epoch=0, lrate=0.100, error=1.000
[1.465489372, 2.362125076, 0]
1.465489372
2.362125076
>epoch=0, lrate=0.100, error=1.000
[3.396561688, 4.400293529, 0]
3.396561688
4.400293529
>epoch=0, lrate=0.100, error=1.000
[1.38807019, 1.850220317, 0]
1.38807019
1.850220317
>epoch=0, lrate=0.100, error=1.000
[3.06407232, 3.005305973, 0]
3.06407232
3.005305973
>epoch=0, lrate=0.100, error=1.000
[7.627531214, 2.759262235, 1]
7.627531214
2.759262235
>epoch=0, lrate=0.100, error=2.000
[5.332441248, 2.088626775, 1]
5.332441248
2.088626775
>epoch=0, lrate=0.100, error=2.000
[6.922596716, 1.77106367, 1]
6.922596716
1.77106367
>epoch=0, lrate=0.100, error=2.000
[8.675418651, -0.242068655, 1]
8.675418651
-0.242068655
>epoch=0, lrate=0.100, error=3.000
[7.673756466, 3.508563011, 1]
7.673756466
3.508563011
>epoch=0, lrate=0.100, error=3.000
[2.7810836, 2.550537003, 0]
2.7810836
2.550537003
>epoch=1, lrate=0.100, error=1.000
[1.465489372, 2.362125076, 0]
1.4